# 앙상블 - 보팅(Voting)

## 01. 보팅(Voting) 개요

일반적으로 단일 학습모델을 사용하면 충분한 성능을 확보하기 어렵습니다.

이러한 부분을 보완하기 위해 **여러 개의 학습모델을 생성하고 그 예측을 결합**함으로써 보다 정확한 최종 예측을 도출하는 기법입니다.

### 앙상블 기법의 종류
- **보팅(Voting)**
- **배깅(Bagging)**
- **부스팅(Boosting)**

### 보팅의 특징
- 다양한 알고리즘을 사용 (예: 로지스틱 + KNN)
- 각각의 알고리즘이 원본 데이터를 그대로 사용하여 각자 분석을 수행
- 결과를 어떻게 종합하느냐에 따라 하드보팅과 소프트보팅으로 구분

### [1] 보팅 방식 구분

#### (1) Hard Voting
- 각 서브샘플에서 예측된 값을 종합하고 **최빈값**으로 최종 예측값을 결정
- 각각의 알고리즘이 예측한 결과값에서 **다수결**로 결정된 값을 최종 결정값으로 정하는 방식

#### (2) Soft Voting
- 각 서브샘플에서 **확률을 계산**하고, 각 확률값을 통해서 최종 예측값을 결정

#### (3) Sklearn에서 지원하는 Voting 알고리즘
- `VotingClassifier` (분류)
- `VotingRegressor` (회귀)

## #01. 준비작업

### [1] 패키지 가져오기

In [1]:
from hossam import *
from pandas import DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve

# 성능 평가 지표 모듈
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)

# 보팅 회귀
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

### [2] 성능 평가 함수

#### VIF 필터링 클래스

In [2]:
class VIFSelector(BaseEstimator, TransformerMixin):
    """VIF 기반 다중공선성 제거 클래스"""

    def __init__(self, threshold=10.0, check_cols=None):
        self.threshold = threshold
        self.check_cols = check_cols
        self.selected_features_ = None

    def fit(self, X, y=None):
        # VIF 계산 및 변수 선택 로직
        # (실제 구현 코드 생략)
        return self

    def transform(self, X):
        # 선택된 변수만 반환
        return X

    def get_feature_names_out(self, input_features=None):
        return self.selected_features_

NameError: name 'BaseEstimator' is not defined

#### 성능평가 함수

In [ ]:
def hs_get_scores(estimator, x_test, y_true):
    """모델 성능 평가 함수"""
    if hasattr(estimator, "named_steps"):
        classname = estimator.named_steps["model"].__class__.__name__
    else:
        classname = estimator.__class__.__name__

    y_pred = estimator.predict(x_test)

    return DataFrame(
        {
            "결정계수(R2)": r2_score(y_true, y_pred),
            "평균절대오차(MAE)": mean_absolute_error(y_true, y_pred),
            "평균제곱오차(MSE)": mean_squared_error(y_true, y_pred),
            "평균오차(RMSE)": np.sqrt(mean_squared_error(y_true, y_pred)),
            "평균 절대 백분오차 비율(MAPE)": mean_absolute_percentage_error(
                y_true, y_pred
            ),
            "평균 비율 오차(MPE)": np.mean((y_true - y_pred) / y_true * 100),
        },
        index=[classname],
    )

#### 과적합 판정 함수

In [ ]:
def hs_learning_cv(
    estimator,
    x,
    y,
    scoring="neg_root_mean_squared_error",
    cv=5,
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1,
):
    """학습 곡선 분석 및 과적합 판정"""
    train_sizes, train_scores, cv_scores = learning_curve(
        estimator=estimator,
        X=x,
        y=y,
        train_sizes=train_sizes,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs,
        shuffle=True,
        random_state=52,
    )

    if hasattr(estimator, "named_steps"):
        classname = estimator.named_steps["model"].__class__.__name__
    else:
        classname = estimator.__class__.__name__

    # neg RMSE → RMSE
    train_rmse = -train_scores
    cv_rmse = -cv_scores

    # 평균 / 표준편차
    train_mean = train_rmse.mean(axis=1)
    cv_mean = cv_rmse.mean(axis=1)
    cv_std = cv_rmse.std(axis=1)

    # 마지막 지점 기준 정량 판정
    final_train = train_mean[-1]
    final_cv = cv_mean[-1]
    final_std = cv_std[-1]

    gap_ratio = final_train / final_cv
    var_ratio = final_std / final_cv

    # 과소적합 기준선
    y_mean = y.mean()
    rmse_naive = np.sqrt(np.mean((y - y_mean) ** 2))
    std_y = y.std()
    min_r2 = 0.10
    rmse_r2 = np.sqrt((1 - min_r2) * np.var(y))
    some_threshold = min(rmse_naive, std_y, rmse_r2)

    # 판정 로직
    if gap_ratio >= 0.95 and final_cv > some_threshold:
        status = "과소적합 (bias 큼)"
    elif gap_ratio <= 0.8:
        status = "과대적합 (variance 큼)"
    elif gap_ratio <= 0.95 and var_ratio <= 0.10:
        status = "일반화 양호"
    elif var_ratio > 0.15:
        status = "데이터 부족 / 분산 큼"
    else:
        status = "판단 유보"

    # 결과 DataFrame
    result_df = DataFrame(
        {
            "Train RMSE": [final_train],
            "CV RMSE 평균": [final_cv],
            "CV RMSE 표준편차": [final_std],
            "Train/CV 비율": [gap_ratio],
            "CV 변동성 비율": [var_ratio],
            "판정 결과": [status],
        },
        index=[classname],
    )

    # 학습곡선 시각화
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, "o-", label="Train RMSE")
    plt.fill_between(train_sizes, train_mean - cv_std, train_mean + cv_std, alpha=0.1)
    plt.plot(train_sizes, cv_mean, "o-", label="CV RMSE")
    plt.fill_between(train_sizes, cv_mean - cv_std, cv_mean + cv_std, alpha=0.1)
    plt.xlabel("Training Size")
    plt.ylabel("RMSE")
    plt.title(f"Learning Curve: {classname}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return result_df

#### 성능평가 + 과적합 판정 동시 수행 함수

In [ ]:
def hs_get_score_cv(
    estimator,
    x_test,
    y_test,
    x_origin,
    y_origin,
    scoring="neg_root_mean_squared_error",
    cv=5,
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1,
):
    """성능평가와 과적합 판정을 동시에 수행"""
    score_df = hs_get_scores(estimator, x_test, y_test)
    cv_df = hs_learning_cv(
        estimator,
        x_origin,
        y_origin,
        scoring=scoring,
        cv=cv,
        train_sizes=train_sizes,
        n_jobs=n_jobs,
    )
    return merge(score_df, cv_df, left_index=True, right_index=True)

### [3] 데이터 가져오기 + 인덱스, 타입변환

**데이터 설명**: 어느 식당의 1년간 일별 매출을 기록한 데이터의 전처리 완료 버전

In [ ]:
origin = load_data("restaurant_sales_preprocessed")
origin.set_index("date", inplace=True)
origin["holiday"] = origin["holiday"].astype("category")
origin["weekend"] = origin["weekend"].astype("category")
origin.info()

### [4] 훈련/검증 데이터 분리

In [ ]:
df = origin
yname = "sales"

x = df.drop(columns=[yname])
y = df[yname]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=52
)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

## #02. Voting 후보 선정

### [1] 지금까지 살펴본 모든 모형들의 성능

| 모델 | 결정계수(R2) | MAE | MSE | RMSE | MAPE | MPE | Train RMSE | CV RMSE 평균 | CV RMSE 표준편차 | Train/CV 비율 | CV 변동성 비율 | 판정 결과 |
|------|-------------|-----|-----|------|------|-----|-----------|-------------|----------------|-------------|--------------|---------|
| Linear | 0.709 | 0.156 | 0.042 | 0.204 | 0.010 | 0.225 | 0.198 | 0.203 | 0.012 | 0.973 | 0.060 | 판단 유보 |
| Ridge | 0.708 | 0.156 | 0.042 | 0.205 | 0.010 | 0.219 | 0.198 | 0.203 | 0.012 | 0.974 | 0.058 | 판단 유보 |
| Lasso | 0.707 | 0.155 | 0.042 | 0.205 | 0.009 | 0.221 | 0.199 | 0.204 | 0.011 | 0.978 | 0.055 | 판단 유보 |
| ElasticNet | 0.709 | 0.156 | 0.042 | 0.204 | 0.010 | 0.223 | 0.198 | 0.203 | 0.012 | 0.974 | 0.059 | 판단 유보 |
| SGD | 0.700 | 0.160 | 0.043 | 0.207 | 0.010 | 0.229 | 0.203 | 0.210 | 0.010 | 0.964 | 0.046 | 판단 유보 |
| KNN | 0.635 | 0.178 | 0.052 | 0.229 | 0.011 | 0.301 | 0.000 | 0.220 | 0.020 | 0.000 | 0.090 | 과대적합 |
| DecisionTree | 0.620 | 0.196 | 0.054 | 0.233 | 0.012 | 0.353 | 0.224 | 0.245 | 0.018 | 0.912 | 0.072 | 일반화 양호 |
| SVR | 0.701 | 0.157 | 0.043 | 0.207 | 0.010 | 0.233 | 0.196 | 0.206 | 0.011 | 0.949 | 0.055 | 일반화 양호 |

### [2] 성능표 데이터프레임

**주의**: 실제로는 이 표를 각 모델을 학습한 결과로부터 얻어야 합니다.
- `hs_get_score_cv()` 함수를 호출한 결과 DataFrame을 `concat()`으로 묶습니다.

In [ ]:
score_df = DataFrame(
    [
        [
            "Linear",
            0.709,
            0.156,
            0.042,
            0.204,
            0.010,
            0.225,
            0.198,
            0.203,
            0.012,
            0.973,
            0.060,
            "판단 유보",
        ],
        [
            "Ridge",
            0.708,
            0.156,
            0.042,
            0.205,
            0.010,
            0.219,
            0.198,
            0.203,
            0.012,
            0.974,
            0.058,
            "판단 유보",
        ],
        [
            "Lasso",
            0.707,
            0.155,
            0.042,
            0.205,
            0.009,
            0.221,
            0.199,
            0.204,
            0.011,
            0.978,
            0.055,
            "판단 유보",
        ],
        [
            "ElasticNet",
            0.709,
            0.156,
            0.042,
            0.204,
            0.010,
            0.223,
            0.198,
            0.203,
            0.012,
            0.974,
            0.059,
            "판단 유보",
        ],
        [
            "SGD",
            0.700,
            0.160,
            0.043,
            0.207,
            0.010,
            0.229,
            0.203,
            0.210,
            0.010,
            0.964,
            0.046,
            "판단 유보",
        ],
        [
            "KNN",
            0.635,
            0.178,
            0.052,
            0.229,
            0.011,
            0.301,
            0.000,
            0.220,
            0.020,
            0.000,
            0.090,
            "과대적합 (variance 큼)",
        ],
        [
            "DecisionTree",
            0.620,
            0.196,
            0.054,
            0.233,
            0.012,
            0.353,
            0.224,
            0.245,
            0.018,
            0.912,
            0.072,
            "일반화 양호",
        ],
        [
            "SVR",
            0.701,
            0.157,
            0.043,
            0.207,
            0.010,
            0.233,
            0.196,
            0.206,
            0.011,
            0.949,
            0.055,
            "일반화 양호",
        ],
    ],
    columns=[
        "모델",
        "결정계수(R2)",
        "MAE",
        "MSE",
        "RMSE",
        "MAPE",
        "MPE",
        "Train RMSE",
        "CV RMSE 평균",
        "CV RMSE 표준편차",
        "Train/CV 비율",
        "CV 변동성 비율",
        "판정 결과",
    ],
)

score_df

### [3] Voting 후보 선정

**선정 기준:**

| 지표 | 역할 | Voting 후보 선정에서 위치 |
|------|------|------------------------|
| Train RMSE | 학습 적합도 | 참고 |
| CV RMSE 평균 | 일반화 성능 | 1순위 |
| CV 변동성 비율 | 안정성 | 1순위 |
| R² | 설명력 | 2순위 |
| RMSE / MAE | 절대 오차 | 2순위 |

In [ ]:
# 1) 과대적합 모델 제거
drop_idx = score_df[score_df["판정 결과"].str.contains("과대적합")].index
candidates = score_df.drop(drop_idx, axis=0).copy()

# 2) Train/CV gap 계산
candidates["Train_CV_gap"] = (candidates["Train/CV 비율"] - 1).abs()

# 3) 1순위 기준 정렬 (일반화 안정성)
candidates = candidates.sort_values(
    by=["CV RMSE 평균", "CV 변동성 비율", "Train_CV_gap"],
    ascending=[True, True, True],
)

# 4) 2순위 기준 정렬 (설명력·에러율)
candidates = candidates.sort_values(
    by=[
        "CV RMSE 평균",
        "CV 변동성 비율",
        "Train_CV_gap",
        "결정계수(R2)",
        "RMSE",
        "MAE",
    ],
    ascending=[True, True, True, False, True, True],
)

# 5) Voting 후보 선정 (상위 3개)
TOP_N = 3
voting_candidates = candidates.head(TOP_N)
voting_model_names = voting_candidates["모델"].tolist()

print(f"선정된 Voting 후보: {voting_model_names}")
voting_candidates.drop(columns="Train_CV_gap").reset_index(drop=True)

### [4] 선정된 후보에 대한 학습 모델 적합

#### (1) Ridge

In [ ]:
%%time
# Ridge 모델 정의
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()), 
    ("model", Ridge(random_state=52))
])

# 하이퍼파라미터 그리드
ridge_param_grid = {"model__alpha": [0.01, 0.1, 1, 10, 100]}

# GridSearchCV
ridge_gs = GridSearchCV(
    estimator=ridge_pipe, 
    param_grid=ridge_param_grid, 
    cv=5, 
    scoring="r2", 
    n_jobs=-1
)

ridge_gs.fit(x_train, y_train)
ridge_estimator = ridge_gs.best_estimator_

# 성능평가
ridge_result_df = hs_get_score_cv(ridge_estimator, x_test, y_test, x, y)
ridge_result_df

#### (2) ElasticNet

In [ ]:
%%time
# ElasticNet 모델 정의
en_pipe = Pipeline([
    ("scaler", StandardScaler()), 
    ("model", ElasticNet(random_state=52))
])

# 하이퍼파라미터 그리드
en_param_grid = {
    "model__alpha": [0.001, 0.01, 0.1, 1, 10],
    "model__l1_ratio": [0.1, 0.5, 0.9],
}

# GridSearchCV
en_gs = GridSearchCV(
    estimator=en_pipe, 
    param_grid=en_param_grid, 
    cv=5, 
    scoring="r2", 
    n_jobs=-1
)

en_gs.fit(x_train, y_train)
en_estimator = en_gs.best_estimator_

# 성능평가
en_result_df = hs_get_score_cv(en_estimator, x_test, y_test, x, y)
en_result_df

#### (3) Linear Regression

In [ ]:
%%time
# Linear Regression 파이프라인
linear_pipe = Pipeline([
    ("vif_selector", VIFSelector(
        check_cols=[
            "visitors", "avg_price", "marketing_cost",
            "delivery_ratio", "rain_mm", "temperature",
        ]
    )),
    ("scaler", StandardScaler()),
    ("model", LinearRegression(n_jobs=-1)),
])

# 선형회귀는 하이퍼파라미터가 없음
linear_param_grid = {}

# GridSearchCV
linear_gs = GridSearchCV(
    estimator=linear_pipe, 
    param_grid=linear_param_grid, 
    cv=5, 
    scoring="r2", 
    n_jobs=-1
)

linear_gs.fit(x_train, y_train)
linear_estimator = linear_gs.best_estimator_

# 성능평가
linear_result_df = hs_get_score_cv(linear_estimator, x_test, y_test, x, y)
linear_result_df

## #03. Voting 모델 적합

### VotingRegressor 학습 및 최적 가중치 탐색

In [ ]:
%%time
# =========================
# 1) VotingRegressor 정의
# =========================
voting_model = VotingRegressor(
    estimators=[
        ("linear", linear_estimator),      # VIF + Scaler + LinearRegression
        ("ridge", ridge_estimator),        # Scaler + Ridge
        ("elasticnet", en_estimator),      # Scaler + ElasticNet
    ],
    weights=[1, 1, 1],
)

# =========================
# 2) Voting 가중치 하이퍼파라미터 그리드
# =========================
voting_param_grid = {
    "weights": [
        [1, 1, 1],
        [2, 1, 1],
        [1, 2, 1],
        [1, 1, 2],
        [2, 2, 1],
        [1, 2, 2],
        [2, 2, 2],
    ]
}

# =========================
# 3) Voting GridSearchCV
# =========================
voting_gs = GridSearchCV(
    estimator=voting_model,
    param_grid=voting_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
)

voting_gs.fit(x_train, y_train)
voting_estimator = voting_gs.best_estimator_
voting_best_weights = voting_gs.best_params_

# =========================
# 4) 성능 평가
# =========================
voting_result_df = hs_get_score_cv(
    voting_estimator,
    x_test,
    y_test,
    x,
    y,
)

print("최적 가중치:")
display(voting_best_weights)
print("\n성능 평가 결과:")
display(voting_result_df)

## #04. 결과 해석

### ⚠️ 중요: Voting의 역할과 해석

**Voting은 "해석 대상 모델"이 아닙니다**

- 해석은 **구성 모델**에서 하고, Voting은 **성능향상 목적**으로만 사용
- 계수·중요도를 해석하는 모델이 아니라 **예측을 평균내는 메타 모델**
- 설명은 각 구성 모델에 대해 개별적으로 수행

### 해석 전략

| 목적 | 사용할 모델 | 방법 |
|------|-----------|------|
| 변수 방향성 | Linear / Ridge | 계수 부호·크기 |
| 변수 선택 | Lasso / ElasticNet | 계수 0 여부 |
| 비선형 영향 | SVR / Tree | SHAP |
| 안정성 확인 | 모든 모델 | CV RMSE |

### 결론
- Voting은 **앙상블을 통한 성능 개선**이 목적
- **모델 해석**은 개별 구성 모델에서 수행
- Voting의 결과는 **최종 예측값**으로만 활용

## 분석 종료

이상으로 Voting 앙상블 기법을 이용한 회귀 모델링을 완료했습니다.

**핵심 요약:**
1. 여러 모델의 예측을 결합하여 성능 향상
2. 과대적합 모델은 제외하고 안정적인 모델만 선정
3. 가중치 튜닝을 통해 최적의 조합 탐색
4. 해석은 개별 모델에서, 성능은 Voting에서 확보